In [12]:
import pandas as pd

# Carga el archivo CSV
df = pd.read_csv("../Data/SemEval/SP_REST_Sb1_TEST.csv", encoding='utf-8')
df

,review,aspect,category,polarity,from,to
0,La comida estuvo muy sabrosa.,comida,FOOD#QUALITY,positive,3,9
1,Quien sea amante de la carne tiene una carta b...,carta,FOOD#STYLE_OPTIONS,positive,39,44
2,Quien sea amante de la carne tiene una carta b...,chuleton,FOOD#STYLE_OPTIONS,negative,99,107
3,Lo único que nos sorprendió es que nos sirvier...,vacio,SERVICE#GENERAL,negative,0,0
4,"El Servicio y la atencion,Impresionante.",Servicio,SERVICE#GENERAL,positive,3,11
...,...,...,...,...,...,...
1067,Estuve comiendo con una amigo y tomamos unos e...,entrantes,FOOD#QUALITY,positive,45,54
1068,"Todo Perfecto y como siempre, el servicio óptimo.",vacio,RESTAURANT#GENERAL,positive,0,0
1069,"Todo Perfecto y como siempre, el servicio óptimo.",servicio,SERVICE#GENERAL,positive,33,41
1070,Es un restaurante seguro para ir y repetir sie...,restaurante,RESTAURANT#GENERAL,positive,6,17


In [13]:
# Elimina las columnas
df = df.drop(['category', 'from', 'to'], axis=1)
df

,review,aspect,polarity
0,La comida estuvo muy sabrosa.,comida,positive
1,Quien sea amante de la carne tiene una carta b...,carta,positive
2,Quien sea amante de la carne tiene una carta b...,chuleton,negative
3,Lo único que nos sorprendió es que nos sirvier...,vacio,negative
4,"El Servicio y la atencion,Impresionante.",Servicio,positive
...,...,...,...
1067,Estuve comiendo con una amigo y tomamos unos e...,entrantes,positive
1068,"Todo Perfecto y como siempre, el servicio óptimo.",vacio,positive
1069,"Todo Perfecto y como siempre, el servicio óptimo.",servicio,positive
1070,Es un restaurante seguro para ir y repetir sie...,restaurante,positive


In [14]:
# Conversión de la columna 'polarity' a numérica
mapping = {'positive': 1, 'negative': 0, 'neutral':2}
df['polarity'] = df['polarity'].map(mapping)

# Elimina las filas con 'polarity_num' igual a 2 (neutral)
df = df[df['polarity'] != 2]
df



,review,aspect,polarity
0,La comida estuvo muy sabrosa.,comida,1
1,Quien sea amante de la carne tiene una carta b...,carta,1
2,Quien sea amante de la carne tiene una carta b...,chuleton,0
3,Lo único que nos sorprendió es que nos sirvier...,vacio,0
4,"El Servicio y la atencion,Impresionante.",Servicio,1
...,...,...,...
1067,Estuve comiendo con una amigo y tomamos unos e...,entrantes,1
1068,"Todo Perfecto y como siempre, el servicio óptimo.",vacio,1
1069,"Todo Perfecto y como siempre, el servicio óptimo.",servicio,1
1070,Es un restaurante seguro para ir y repetir sie...,restaurante,1


In [15]:
# Reemplaza valores en la columna 'review'
df['review'] = df['review'].str.replace('9 Reinas', 'nueveReina')
df['review'] = df['review'].str.replace('9R', 'nueveReina', )
df['review'] = df['review'].str.replace('L´Olive', 'LOlive')
df['review'] = df['review'].str.replace('cena-espectáculo', 'cena') #Probar metricas
df['review'] = df['review'].str.replace('calçots', 'calcots')
df['review'] = df['review'].str.replace('calçotada', 'calcotada')
df['review'] = df['review'].str.replace('calçots', 'calcots')

# Reemplaza valores en la columna 'aspect'
df['aspect'] = df['aspect'].replace('9 Reinas', 'nueveReina')
df['aspect'] = df['aspect'].replace('9R', 'nueveReina')
df['aspect'] = df['aspect'].replace('L´Olive', 'LOlive')
df['aspect'] = df['aspect'].replace('cena-espectáculo', 'cena') #Probar metricas
df['aspect'] = df['aspect'].replace('calçots', 'calcots')
df['aspect'] = df['aspect'].replace('calçotada', 'calcotada')
df['aspect'] = df['aspect'].replace('calçots', 'calcots')

C:\Users\Miguel\AppData\Local\Temp\ipykernel_6588\1532675631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.replace('9 Reinas', 'nueveReina')
C:\Users\Miguel\AppData\Local\Temp\ipykernel_6588\1532675631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.replace('9R', 'nueveReina', )
C:\Users\Miguel\AppData\Local\Temp\ipykernel_6588\1532675631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [16]:
# Guarda el DataFrame preprocesado (opcional)
df.to_csv("tu_archivo_preprocesado.csv", index=False, encoding='utf-8', sep=';')

### Añadir columna segmento

In [17]:
from os import sep
import spacy
import re
from nltk.stem import PorterStemmer
import xml.etree.ElementTree as ET
import csv
stemmer = PorterStemmer() # Inicializa un stemmer de Porter para reducir palabras a su raíz

# Cargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm") # Carga un modelo de procesamiento de lenguaje natural en español


# Dada la oración y el aspecto, extrae el segmento donde se encuentra el aspecto
def extraer_segmento(oracion,aspecto):
    """
    Extrae el segmento de una oración que contiene un aspecto dado. Combina varias funciones para identificar y retornar la porción relevante de texto.
    """
    dependencias = extraer_dependencias(oracion, aspecto) # Obtiene las dependencias sintácticas del aspecto en la oración
    lista = palabras_sin_repetir(dependencias) # Elimina las palabras repetidas de la lista de dependencias (función no definida aquí)
    segmento = combinacion(oracion,lista) # Combina las palabras en un segmento coherente (función no definida aquí)

    return convertir_lista_oracion(segmento) # Convierte la lista de palabras resultante en una oración (función no definida aquí)


def extraer_dependencias(texto_opinion, palabra_clave):
    """
    Extrae información sintáctica relacionada con una palabra clave en un texto. Identifica y devuelve frases relacionadas con el aspecto.
    """
    doc_analizado = nlp(texto_opinion) # Analiza el texto usando el modelo de spaCy
    frases_extraidas = [] # Inicializa una lista para guardar las frases extraídas

    for palabra_actual in doc_analizado: # Itera sobre cada palabra del texto analizado
        if palabra_actual.text == palabra_clave or palabra_actual.text == stemmer.stem(palabra_clave) : # Busca la palabra clave o su raíz

            palabras_asociadas = [] # Lista para guardar las palabras asociadas a la palabra clave

            # Extrae palabras a la izquierda y derecha de la palabra clave
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_actual, 'left'))
            palabras_asociadas.append(palabra_actual.text)
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_actual, 'right'))

            palabra_principal = palabra_actual.head # Encuentra la palabra principal (la raíz sintáctica) de la palabra clave

            # Extrae sujetos y objetos directos relacionados con la palabra principal
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_principal, 'left'))
            palabras_asociadas.append(palabra_principal.text)
            palabras_asociadas.extend(extraer_palabras_adyacentes(palabra_principal, 'right'))

            # Agrega n-gramas, contexto y la frase extraída a la lista de frases
            frases_extraidas.append(palabras_asociadas)
            #frases_extraidas.append(n_gramas(texto_opinion, palabra_clave, 6))
            frases_extraidas.append(extraccion_contexto(texto_opinion, palabra_clave))

    return frases_extraidas # Devuelve la lista de frases extraídas


# Funciones auxiliares para extraer dependencias

def extraer_palabras_adyacentes(token_actual, direccion):
    """
    Extrae las palabras a la izquierda o derecha de un token, incluyendo adjetivos relacionados con sustantivos.

    Args:
        token_actual: El token de referencia.
        direccion: 'left' para palabras a la izquierda, 'right' para palabras a la derecha.

    Returns:
        Una lista de palabras.
    """
    palabras = []
    adyacentes = token_actual.lefts if direccion == 'left' else token_actual.rights
    for token_adyacente in adyacentes:
        palabras.append(token_adyacente.text)
        if token_adyacente.dep_ == 'NOUN':
            for token_adjetivo in token_adyacente.lefts:
                if token_adjetivo.dep_ == 'ADJ':
                    palabras.append(token_adjetivo.text)
            for token_adjetivo in token_adyacente.rights:
                if token_adjetivo.dep_ == 'ADJ':
                    palabras.append(token_adjetivo.text)
    return palabras

def n_gramas(oracion, palabra, numero):
    """
    Extrae n-gramas (secuencias de n palabras) alrededor de una palabra clave en una oración.
    """
    palabras = limpiar_oracion(oracion).split() # Limpia la oración y la divide en palabras 
    index_palabra = [] # Inicializa una lista para guardar el índice de la palabra clave en la lista de palabras

    for index, word in enumerate(palabras): # Itera sobre las palabras para encontrar la palabra clave
        if word.lower() == palabra.lower() or word.lower() == stemmer.stem(palabra.lower()): # Busca la palabra clave o su raíz
            index_palabra = index
            break

    palabras_anteriores = palabras[max(0, index_palabra - numero):index_palabra] # Extrae las palabras anteriores a la palabra clave
    palabras_posteriores = palabras[index_palabra + 1:index_palabra + 1 + numero] # Extrae las palabras posteriores a la palabra clave

    return palabras_anteriores + [palabra] + palabras_posteriores # Devuelve una lista con las palabras anteriores, la palabra clave y las palabras posteriores


def extraccion_contexto(oracion, palabra_relevante):
    """
    Extrae el contexto de una palabra clave en una oración, incluyendo modificadores.
    """
    doc = nlp(oracion) # Analiza la oración con spaCy
    token_palabra_relevante = None # Inicializa la variable para guardar el token de la palabra clave
    for token in doc: # Busca el token de la palabra clave en la oración analizada
        if token.text.lower() == palabra_relevante:
            token_palabra_relevante = token
            break

    if not token_palabra_relevante: # Si no se encuentra la palabra clave
        return "Palabra relevante no encontrada en la oración"

    modificadores = [] # Inicializa una lista para guardar los modificadores de la palabra clave
    for token in doc: # Itera sobre los tokens de la oración analizada
        # Busca modificadores de la palabra clave (adjetivos, adverbios, etc.)
        if token.head == token_palabra_relevante and token.dep_ in ["amod", "advmod", "nsubj", "nummod", "det"]: 
            modificadores.append(token.text)

    return modificadores # Devuelve la lista de modificadores


def limpiar_oracion(oracion):

    # Usar una expresión regular para encontrar y reemplazar todos los caracteres que no sean letras
    oracion_limpia = re.sub(r'[^a-zA-Z\sáéíóúüÁÉÍÓÚÜñÑ]', ' ', oracion) 
    # Devolver la oración limpia
    return oracion_limpia

def palabras_sin_repetir(lista_de_listas):
    palabras = set()  # Utilizamos un conjunto para evitar palabras duplicadas  
    for sublista in lista_de_listas:
        for palabra in sublista:
            palabras.add(palabra) 

    return list(palabras)  # Convertimos el conjunto de palabras de nuevo a una lista y la devolvemos

def combinacion(oracion, lista_palabras):
    """
    Extrae las palabras de una oración que están presentes en una lista dada, 
    evitando duplicados y manejando algunas variaciones simples.

    Args:
        oracion: La oración de entrada como una cadena de texto.
        lista_palabras: Una lista de palabras a buscar en la oración.

    Returns:
        Una lista de palabras de la oración que se encontraron en la lista de palabras, sin duplicados.
    """
    resultado = [] # Inicializa una lista vacía para almacenar las palabras encontradas.
    tokens = re.split(r'[,\s]+', oracion)
    for token in tokens: # Itera sobre cada palabra (token) en la oración.
        if (token in lista_palabras) or (token[0:-1] in lista_palabras): # Verifica si el token o su forma sin el último carácter está en la lista de palabras.
            if not token in resultado: # Verifica si el token ya está en la lista de resultados para evitar duplicados.
                resultado.append(token) # Agrega el token a la lista de resultados.

    return resultado # Devuelve la lista de palabras encontradas.

def convertir_lista_oracion(lista_palabras):

    oracion = " ".join(lista_palabras)  # Unir las palabras con espacios
    return oracion


In [18]:
import pandas as pd

def procesar_csv(ruta_archivo):

    df = pd.read_csv(ruta_archivo, sep=";")
    df["Segmento"] = ""
    lista_segmentos = []
    try:
        for index, fila in df.iterrows():
            texto = fila['review']
            aspecto = fila['aspect']
            tokens = aspecto.split()
            if aspecto == " vacio":
                segmento = texto
            else:
                segmento = extraer_segmento(texto,tokens[0])
                
            lista_segmentos.append(segmento)

    except FileNotFoundError:
        print(f"Error: Archivo '{ruta_archivo}' no encontrado.")
    except Exception as e:
        print(f"Error al procesar el archivo: {e}")

    df["Segmento"] = lista_segmentos
    df.to_csv(ruta_archivo, index=False, sep=";")


# Reemplaza 'tu_archivo.csv' con la ruta real a tu archivo CSV
ruta_archivo = '../Preprocesing/tu_archivo_preprocesado.csv'  
procesar_csv(ruta_archivo)


### Añadir ventana

In [11]:
import pandas as pd

def extraer_segmento(review, aspecto, ventana_size=6):
    """Extrae un segmento de texto alrededor de un aspecto dado."""
    try:
        indice = review.split().index(aspecto) # Encuentra el índice del aspecto
        inicio = max(0, indice - ventana_size)
        fin = min(len(review.split()), indice + ventana_size + 1)
        segmento = " ".join(review.split()[inicio:fin])
        return segmento
    except ValueError: # Si el aspecto no se encuentra en el review
        return review


# Carga el archivo CSV
df = pd.read_csv("../Preprocesing/tu_archivo_preprocesado.csv", sep=";")

# Aplica la función para crear la nueva columna
df["ventana"] = df.apply(lambda row: extraer_segmento(row["review"], row["aspect"]), axis=1)

# Guarda el DataFrame actualizado
df.to_csv("ventana6.csv", sep=";", index=False)

print("Archivo CSV actualizado correctamente.")



Archivo CSV actualizado correctamente.
